In [1]:
#-*-coding:utf-8-*-
import util
import tensorflow as tf
MNIST_CLASSIFIER_FROZEN_GRAPH = './classify_mnist_graph_def.pb'
INPUT_TENSOR = 'inputs:0'
OUTPUT_TENSOR = 'logits:0'
def EuclideanDistances(A, B):

    BT = B.transpose()
    # vecProd = A * BT
    vecProd = np.dot(A, BT)
    # print(vecProd)
    SqA = A ** 2
    # print(SqA)
    sumSqA = np.matrix(np.sum(SqA, axis=1))
    sumSqAEx = np.tile(sumSqA.transpose(), (1, vecProd.shape[1]))
    # print(sumSqAEx)

    SqB = B ** 2
    sumSqB = np.sum(SqB, axis=1)
    sumSqBEx = np.tile(sumSqB, (vecProd.shape[0], 1))
    SqED = sumSqBEx + sumSqAEx - 2 * vecProd
    SqED[SqED < 0] = 0.0
    ED = np.sqrt(SqED)
    return np.divide(ED.sum(),ED.shape[0]*ED.shape[1])
def cal_distance_image_real(images,labels):
    eval_images=tf.convert_to_tensor(images)
    y_logits=util.mnist_logits(eval_images, MNIST_CLASSIFIER_FROZEN_GRAPH, INPUT_TENSOR, OUTPUT_TENSOR)
    y_logits=tf.Session().run(y_logits)
    dict={}
    all_dis=[]
    for i in range(10):
        dict[i]=[]
    for i in range(len(labels)):
        dict[labels[i]].append(y_logits[i])
    for i in range(10):
        dict[i]=np.array(dict[i])
        if len(dict[i]):
            dis = EuclideanDistances(dict[i], dict[i])  # 生成图片的紧度
        else:
            dis = -1
        all_dis.append(dis)
    return np.array(all_dis)
def cal_distance_image_fake(images):
    eval_images=tf.convert_to_tensor(images)
    y_logits=util.mnist_logits(eval_images, MNIST_CLASSIFIER_FROZEN_GRAPH, INPUT_TENSOR, OUTPUT_TENSOR)
    y_logits=tf.Session().run(y_logits)
    labels = tf.Session().run(tf.argmax(y_logits, 1))
    dict={}
    all_dis=[]
    for i in range(10):
        dict[i]=[]
    for i in range(len(labels)):
        dict[labels[i]].append(y_logits[i])
    for i in range(10):
        dict[i]=np.array(dict[i])
        if len(dict[i]):
            dis = EuclideanDistances(dict[i], dict[i])  # 生成图片的紧度
        else:
            dis = -1
        all_dis.append(dis)
    return np.array(all_dis)

    

In [2]:
import os
if not os.path.isdir('saved_models_mnist_{}'.format('cgan')):
    os.mkdir('saved_models_mnist_{}'.format('cgan'))
f = open('saved_models_mnist_{}/log_collapse1.txt'.format('cgan'), mode='w')


from keras.datasets import fashion_mnist,mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply
from keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
import matplotlib.pyplot as plt

import numpy as np
import os
class CGAN():
    def __init__(self):
        # Input shape
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.num_classes = 10
        self.latent_dim = 100
        self.x = []
        self.y = np.zeros((31, 1), dtype=np.int)
        self.y = list(self.y)
        for i in range(31):
            self.y[i] = []

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss=['binary_crossentropy'],
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise and the target label as input
        # and generates the corresponding digit of that label
        noise = Input(shape=(self.latent_dim,))
        label = Input(shape=(1,))
        img = self.generator([noise, label])

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated image as input and determines validity
        # and the label of that image
        valid = self.discriminator([img, label])

        # The combined model  (stacked generator and discriminator)
        # Trains generator to fool discriminator
        self.combined = Model([noise, label], valid)
        self.combined.compile(loss=['binary_crossentropy'],
            optimizer=optimizer)

    def build_generator(self):

        model = Sequential()

        model.add(Dense(512 * 7 * 7, input_dim=self.latent_dim))
        model.add(Reshape((7,7, 512)))
        model.add(Activation("relu"))

        model.add(BatchNormalization(momentum=0.8))
        model.add(UpSampling2D())
        model.add(Conv2D(256, kernel_size=3, padding="same"))
        model.add(Activation("relu"))

        model.add(BatchNormalization(momentum=0.8))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=3,strides=1,padding="same"))
        model.add(Activation("relu"))

        model.add(BatchNormalization(momentum=0.8))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=3,strides=1, padding="same"))
        model.add(Activation("relu"))

        model.add(Conv2D(self.channels, strides=2,kernel_size=3, padding="same"))
        model.add(Activation("tanh"))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        label = Input(shape=(1,), dtype='int32')
        label_embedding = Flatten()(Embedding(self.num_classes, self.latent_dim)(label))

        model_input = multiply([noise, label_embedding])
        img = model(model_input)

        return Model([noise, label], img)
    def build_discriminator(self):
        model = Sequential()
        model.add(Dense(self.img_rows*self.img_cols*self.channels, input_dim=np.prod(self.img_shape)))
        model.add(Reshape((self.img_rows,self.img_cols,self.channels)))

        model.add(Conv2D(64, kernel_size=3, strides=2,padding="same"))
        model.add(LeakyReLU(alpha=0.2))

        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        model.add(LeakyReLU(alpha=0.2))

        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(256, kernel_size=3, strides=2, padding="same"))
        model.add(LeakyReLU(alpha=0.2))

        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(512, kernel_size=3, strides=1, padding="same"))
        model.add(LeakyReLU(alpha=0.2))

        model.add(Flatten())
        model.add(Dense(1, activation='sigmoid'))

        model.summary()
        img = Input(shape=self.img_shape)
        label = Input(shape=(1,), dtype='int32')

        label_embedding = Flatten()(Embedding(self.num_classes, np.prod(self.img_shape))(label))
        flat_img = Flatten()(img)

        model_input = multiply([flat_img, label_embedding])

        validity = model(model_input)

        return Model([img, label], validity)

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        (X_train, y_train), (X_test, y_test) = mnist.load_data()

        # Configure input
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_train = np.expand_dims(X_train, axis=3)
        X_test = (X_test.astype(np.float32) - 127.5) / 127.5
        # X_test = X_test / 127.5 - 1.
        X_test = np.expand_dims(X_test, axis=3)

        y_train = y_train.reshape(-1, 1)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        nb_batches = int(X_train.shape[0] / batch_size)
        global_step = 0

        for epoch in range(epochs):

            for index in range(nb_batches):
                global_step += 1
                imgs = X_train[index * batch_size:(index + 1) * batch_size]
                labels = y_train[index * batch_size:(index + 1) * batch_size]

                # Sample noise as generator input
                noise = np.random.normal(0, 1, (batch_size, 100))

                # Generate a half batch of new images
                gen_imgs = self.generator.predict([noise, labels])

                # Train the discriminator
                d_loss_real = self.discriminator.train_on_batch([imgs, labels], valid)
                d_loss_fake = self.discriminator.train_on_batch([gen_imgs, labels], fake)
                d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

                # ---------------------
                #  Train Generator
                # ---------------------

                # Condition on labels
                sampled_labels = np.random.randint(0, 10, batch_size).reshape(-1, 1)

                # Train the generator
                g_loss = self.combined.train_on_batch([noise, sampled_labels], valid)

                # Plot the progress
                print("epoch:%d step:%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch,global_step, d_loss[0], 100 * d_loss[1], g_loss))

                sampleSize = 5000
                # If at save interval => save generated image samples
                if global_step % sample_interval == 0:
                    s = self.sample_images(global_step, X_test,y_test, sampleSize)
    def sample_images(self, epoch,x_test,y_test,sample_num):
        r, c = sample_num//10, 10
        noise = np.random.normal(0, 1, (r * c, 100))
        sampled_labels = np.array([num for _ in range(r) for num in range(c)])
        gen_imgs = self.generator.predict([noise,sampled_labels])
        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5
        labels = np.squeeze(y_test[:sample_num])
        labels = np.squeeze(labels)
        dis_real = cal_distance_image_real(x_test[:sample_num], labels)
        dis_fake = cal_distance_image_fake(gen_imgs)
        dis_cha = dis_real - dis_fake
        print('##############')
        # print(dis_real)
        # print(dis_fake)
        print(dis_cha)
        print('##########')
        f.writelines('\n')
        f.writelines('epoch:' + str(epoch))
        f.writelines('\n')
        f.writelines('紧度')
        f.writelines('\n')
        f.writelines(' %.8f ' % (i) for i in dis_cha)
        f.writelines('\n')


if __name__ == '__main__':
    cgan = CGAN()
    cgan.train(epochs=30, batch_size=64, sample_interval=200)

Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 784)               615440    
_________________________________________________________________
reshape_1 (Reshape)          (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 64)        640       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 14, 14, 64)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 14, 14, 64)        256       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 128)         73856     
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 7, 7, 128)         0         
__________

/home/imi432_006/anaconda3/envs/tf/lib/python3.5/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


epoch:0 step:1 [D loss: 1.234361, acc.: 21.09%] [G loss: 0.187136]
epoch:0 step:2 [D loss: 1.626225, acc.: 50.00%] [G loss: 0.371882]
epoch:0 step:3 [D loss: 0.839789, acc.: 60.94%] [G loss: 1.734611]
epoch:0 step:4 [D loss: 0.802841, acc.: 55.47%] [G loss: 1.941102]
epoch:0 step:5 [D loss: 0.823138, acc.: 46.09%] [G loss: 2.343789]
epoch:0 step:6 [D loss: 0.521292, acc.: 75.78%] [G loss: 2.581154]
epoch:0 step:7 [D loss: 0.535790, acc.: 73.44%] [G loss: 2.141510]
epoch:0 step:8 [D loss: 0.801034, acc.: 51.56%] [G loss: 2.974207]
epoch:0 step:9 [D loss: 0.592668, acc.: 67.19%] [G loss: 3.432101]
epoch:0 step:10 [D loss: 0.510780, acc.: 71.09%] [G loss: 3.015180]
epoch:0 step:11 [D loss: 1.023798, acc.: 67.97%] [G loss: 3.135301]
epoch:0 step:12 [D loss: 0.918855, acc.: 63.28%] [G loss: 4.431047]
epoch:0 step:13 [D loss: 0.742997, acc.: 75.00%] [G loss: 4.041420]
epoch:0 step:14 [D loss: 0.502613, acc.: 71.09%] [G loss: 3.551806]
epoch:0 step:15 [D loss: 0.406933, acc.: 81.25%] [G loss:

epoch:0 step:125 [D loss: 0.440912, acc.: 85.94%] [G loss: 7.907975]
epoch:0 step:126 [D loss: 0.420633, acc.: 91.41%] [G loss: 6.786548]
epoch:0 step:127 [D loss: 0.121031, acc.: 95.31%] [G loss: 3.746032]
epoch:0 step:128 [D loss: 0.269543, acc.: 91.41%] [G loss: 6.812435]
epoch:0 step:129 [D loss: 0.368960, acc.: 85.94%] [G loss: 4.759154]
epoch:0 step:130 [D loss: 0.126483, acc.: 93.75%] [G loss: 6.491962]
epoch:0 step:131 [D loss: 0.086315, acc.: 96.88%] [G loss: 5.841400]
epoch:0 step:132 [D loss: 0.457604, acc.: 82.81%] [G loss: 3.605254]
epoch:0 step:133 [D loss: 0.121357, acc.: 97.66%] [G loss: 4.981948]
epoch:0 step:134 [D loss: 0.082350, acc.: 98.44%] [G loss: 5.931878]
epoch:0 step:135 [D loss: 0.153307, acc.: 95.31%] [G loss: 4.438735]
epoch:0 step:136 [D loss: 0.678324, acc.: 68.75%] [G loss: 7.178815]
epoch:0 step:137 [D loss: 0.572998, acc.: 78.12%] [G loss: 6.313098]
epoch:0 step:138 [D loss: 0.332341, acc.: 85.16%] [G loss: 3.510972]
epoch:0 step:139 [D loss: 1.174368

...

epoch:29 step:27580 [D loss: 0.096667, acc.: 97.66%] [G loss: 3.110282]
epoch:29 step:27581 [D loss: 0.247199, acc.: 95.31%] [G loss: 2.130744]
epoch:29 step:27582 [D loss: 0.756249, acc.: 58.59%] [G loss: 2.674366]
epoch:29 step:27583 [D loss: 0.421535, acc.: 83.59%] [G loss: 2.951553]
epoch:29 step:27584 [D loss: 1.315048, acc.: 49.22%] [G loss: 1.992198]
epoch:29 step:27585 [D loss: 0.497852, acc.: 75.00%] [G loss: 1.517661]
epoch:29 step:27586 [D loss: 0.615973, acc.: 62.50%] [G loss: 2.808438]
epoch:29 step:27587 [D loss: 0.346050, acc.: 86.72%] [G loss: 1.776934]
epoch:29 step:27588 [D loss: 0.530757, acc.: 71.88%] [G loss: 0.807665]
epoch:29 step:27589 [D loss: 0.546445, acc.: 73.44%] [G loss: 1.758227]
epoch:29 step:27590 [D loss: 0.306990, acc.: 89.06%] [G loss: 0.569852]
epoch:29 step:27591 [D loss: 0.263502, acc.: 95.31%] [G loss: 1.427411]
epoch:29 step:27592 [D loss: 0.905262, acc.: 58.59%] [G loss: 2.068725]
epoch:29 step:27593 [D loss: 0.608834, acc.: 63.28%] [G loss: 2.

epoch:29 step:27692 [D loss: 0.446195, acc.: 81.25%] [G loss: 1.925547]
epoch:29 step:27693 [D loss: 0.335381, acc.: 90.62%] [G loss: 1.875092]
epoch:29 step:27694 [D loss: 0.364879, acc.: 85.94%] [G loss: 1.697008]
epoch:29 step:27695 [D loss: 0.222337, acc.: 98.44%] [G loss: 1.674763]
epoch:29 step:27696 [D loss: 0.300022, acc.: 94.53%] [G loss: 2.023256]
epoch:29 step:27697 [D loss: 0.366325, acc.: 82.81%] [G loss: 1.240672]
epoch:29 step:27698 [D loss: 0.626909, acc.: 60.16%] [G loss: 1.523674]
epoch:29 step:27699 [D loss: 0.494133, acc.: 75.00%] [G loss: 1.988567]
epoch:29 step:27700 [D loss: 0.210198, acc.: 92.19%] [G loss: 1.684340]
epoch:29 step:27701 [D loss: 0.433819, acc.: 83.59%] [G loss: 1.558133]
epoch:29 step:27702 [D loss: 0.394934, acc.: 86.72%] [G loss: 1.991256]
epoch:29 step:27703 [D loss: 0.282301, acc.: 93.75%] [G loss: 2.004594]
epoch:29 step:27704 [D loss: 0.651855, acc.: 65.62%] [G loss: 1.466260]
epoch:29 step:27705 [D loss: 0.456866, acc.: 79.69%] [G loss: 1.

epoch:29 step:27804 [D loss: 0.603285, acc.: 67.19%] [G loss: 3.228807]
epoch:29 step:27805 [D loss: 0.078595, acc.: 99.22%] [G loss: 2.286410]
epoch:29 step:27806 [D loss: 0.055490, acc.: 100.00%] [G loss: 2.064325]
epoch:29 step:27807 [D loss: 0.201496, acc.: 91.41%] [G loss: 2.698651]
epoch:29 step:27808 [D loss: 0.074551, acc.: 98.44%] [G loss: 1.918831]
epoch:29 step:27809 [D loss: 0.302560, acc.: 85.94%] [G loss: 2.547446]
epoch:29 step:27810 [D loss: 0.049349, acc.: 99.22%] [G loss: 3.609567]
epoch:29 step:27811 [D loss: 0.716894, acc.: 64.06%] [G loss: 3.012720]
epoch:29 step:27812 [D loss: 1.477018, acc.: 49.22%] [G loss: 3.032691]
epoch:29 step:27813 [D loss: 1.425384, acc.: 50.78%] [G loss: 2.078026]
epoch:29 step:27814 [D loss: 0.688756, acc.: 61.72%] [G loss: 1.821605]
epoch:29 step:27815 [D loss: 0.785217, acc.: 53.12%] [G loss: 1.415463]
epoch:29 step:27816 [D loss: 0.205820, acc.: 96.09%] [G loss: 1.714926]
epoch:29 step:27817 [D loss: 0.127205, acc.: 97.66%] [G loss: 2

epoch:29 step:27920 [D loss: 0.404474, acc.: 80.47%] [G loss: 2.340559]
epoch:29 step:27921 [D loss: 0.835162, acc.: 51.56%] [G loss: 1.197231]
epoch:29 step:27922 [D loss: 0.335184, acc.: 85.16%] [G loss: 1.705412]
epoch:29 step:27923 [D loss: 0.499912, acc.: 74.22%] [G loss: 1.447661]
epoch:29 step:27924 [D loss: 0.601454, acc.: 63.28%] [G loss: 1.093785]
epoch:29 step:27925 [D loss: 0.394317, acc.: 88.28%] [G loss: 1.507193]
epoch:29 step:27926 [D loss: 0.426115, acc.: 86.72%] [G loss: 1.648206]
epoch:29 step:27927 [D loss: 0.178200, acc.: 95.31%] [G loss: 2.119148]
epoch:29 step:27928 [D loss: 0.104229, acc.: 99.22%] [G loss: 1.506753]
epoch:29 step:27929 [D loss: 0.200884, acc.: 94.53%] [G loss: 1.946559]
epoch:29 step:27930 [D loss: 0.226833, acc.: 96.88%] [G loss: 2.230098]
epoch:29 step:27931 [D loss: 0.363251, acc.: 90.62%] [G loss: 0.540097]
epoch:29 step:27932 [D loss: 0.614105, acc.: 65.62%] [G loss: 1.850602]
epoch:29 step:27933 [D loss: 0.635093, acc.: 61.72%] [G loss: 1.

epoch:29 step:28032 [D loss: 0.170060, acc.: 95.31%] [G loss: 2.354536]
epoch:29 step:28033 [D loss: 0.575203, acc.: 71.09%] [G loss: 2.856393]
epoch:29 step:28034 [D loss: 0.432965, acc.: 79.69%] [G loss: 2.420849]
epoch:29 step:28035 [D loss: 0.893093, acc.: 53.12%] [G loss: 1.784527]
epoch:29 step:28036 [D loss: 0.774312, acc.: 60.16%] [G loss: 1.264139]
epoch:29 step:28037 [D loss: 0.646690, acc.: 69.53%] [G loss: 1.506698]
epoch:29 step:28038 [D loss: 0.675158, acc.: 62.50%] [G loss: 1.466416]
epoch:29 step:28039 [D loss: 0.739588, acc.: 56.25%] [G loss: 1.589679]
epoch:29 step:28040 [D loss: 0.837305, acc.: 50.78%] [G loss: 1.349720]
epoch:29 step:28041 [D loss: 0.429481, acc.: 80.47%] [G loss: 1.384221]
epoch:29 step:28042 [D loss: 0.862974, acc.: 46.09%] [G loss: 1.107692]
epoch:29 step:28043 [D loss: 0.309740, acc.: 89.06%] [G loss: 1.197692]
epoch:29 step:28044 [D loss: 0.438956, acc.: 81.25%] [G loss: 1.367101]
epoch:29 step:28045 [D loss: 0.793973, acc.: 46.88%] [G loss: 1.